## PREPROCESSING

Ricette definite in un formato semistrutturato, raggruppamento degli step e ingredienti in un unica stringa, funzioni di utilità

### Funzioni util e import

In [ ]:
from pathlib import Path
import pandas as pd
from IPython.display import display
from ast import literal_eval
import re
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
from nltk.sem import relextract
import spacy
from spacy import displacy
from nltk.corpus import conll2000
from spacy.symbols import X, NUM, VERB, NOUN, ADP
from nltk.corpus import wordnet as wn

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('conll2000')


!python -m spacy download en_core_web_md

NLP = spacy.load('en_core_web_md')


def string_recipe(i):
    return dataset.iloc[i]['title'] + "\n\n" + dataset.iloc[i]['ingredients'] + "\n\n" + dataset.iloc[i]['step'] 

### Caricamento del dataset

In [ ]:
dataset = pd.read_csv(
    Path("../data/test_dataset.csv").resolve(), 
    index_col=[0], 
    names=["index", "title","ingredients","step"], 
    usecols=[0,1,2,3]
    )

for index in range(len(dataset)):
    dataset.iloc[index]['ingredients'] = ".\n".join(literal_eval(dataset.iloc[index]['ingredients']))
    dataset.iloc[index]['step'] = " ".join(literal_eval(dataset.iloc[index]['step']))

display(dataset.head())
print(string_recipe(10))


### Estrazione delle abbreviazioni

Fase iniziale di ritrovamento del set di abbreviazioni

In [ ]:
abbrv_dataset = pd.read_csv(
    Path("../data/test_dataset.csv").resolve(), 
    index_col=[0], 
    names=["index", "title","ingredients","step"], 
    usecols=[0,1,2,3]
    )

abbrv = set()
for index in range(len(dataset)):
    abbrv_dataset.iloc[index]['ingredients'] = " ".join(literal_eval(abbrv_dataset.iloc[index]['ingredients']))
    for element in re.findall(r"[A-Za-z]*\.", abbrv_dataset.iloc[index]['ingredients']):
        abbrv.add(element)
    
print(abbrv)

Rimozione delle abbreviazioni in quanto possono essere dannose per il processo di tokenizzazione. Es pkg. ---> package

In [ ]:
def expand_abbreviations(ingredients_string):
    __ABBREVIATIONS__ = {
        'pkg.'  :   'package',
        'tsb.'  :   'tablespoon',
        'no.'   :   'number',
        'pt.'   :   'pint',
        'no.'   :   'number',
        'gal.'  :   'gallon',
        'tbsp.' :   'tablespoon',
        'sq.'   :   'square',
        'oz.'   :   'ounce',
        'lb.'   :   'pound',
        'qt.'   :   'quart',
        'c.'    :   'cup',
        'tsp.'  :   'teaspoon'
    }
    for item, value in __ABBREVIATIONS__.items():
        ingredients_string = ingredients_string.lower().replace(item, value)
    return ingredients_string


for index in range(len(dataset)):
    dataset.iloc[index]['ingredients'] = expand_abbreviations(dataset.iloc[index]['ingredients'])

### Sentence splitting
Il contenuto delle colonne 'ingredients' e 'step' verrà suddiviso in frasi. In precedenza i periodi contenuti nelle singole celle sono stati formattati in modo tale da renderli riconoscibili e facilmente suddivisibili in frasi ben separate.

In [ ]:
for index in range(len(dataset)):
     dataset.iloc[index]['ingredients'] = (sent_tokenize(dataset.iloc[index]['ingredients']))
print(dataset.iloc[10]['ingredients'])

In [ ]:
for index in range(len(dataset)):
     dataset.iloc[index]['step'] = (sent_tokenize(dataset.iloc[index]['step']))
print(dataset.iloc[10]['step'])

### Rimozione quantità doppie

In [ ]:
dr_reg = r'\d*\s*\(.*\)'

for index in range(len(dataset)):
    for value in range(len(dataset.iloc[index]['ingredients'])):
        elements = re.findall(dr_reg, dataset.iloc[index]['ingredients'][value])
        for e in elements:
            new_string = dataset.iloc[index]['ingredients'][value].replace(e,e[e.find("(")+1: e.find(")")].strip())
            dataset.iloc[index]['ingredients'][value] = new_string

## ANALISI

### Stop words removal
Nella colonna 'step' troviamo una serie di passaggi da compiere per creare la ricetta. Questi passaggi sono scritti in linguaggio naturale e possono essere semplificati rimuovendo delle parole dette stop words.

In [ ]:
# OPTINAL -> Risultati in swr_dataset"

stop_words = set(stopwords.words('english'))
swr_dataset = dataset.copy(deep=True)

tk = lambda x,st: ' '.join([w for w in x if w not in st])
for index in range(len(dataset)):
    swr_dataset.iloc[index]['step'] = [tk(word_tokenize(sent), stop_words) for sent in dataset.iloc[index]['step']]

### Stemming e Lemming
Questi due processi potrebbero portare valore all'analisi del dominio. Il codice per entrambi viene proposto qui.

In [ ]:
# OPTINAL -> Risultati in lem_dataset"

lemmatizer = WordNetLemmatizer()
lem_dataset = swr_dataset.copy(deep=True)

for index in range(len(swr_dataset)):
    lem_dataset.iloc[index]['step'] = stem_sent = [' '.join([lemmatizer.lemmatize(w) for w in word_tokenize(sent)]) for sent in swr_dataset.iloc[index]['step']]

In [ ]:
# OPTINAL -> Risultati in stm_dataset"

stemmer = PorterStemmer()
stm_dataset = lem_dataset.copy(deep=True)

for index in range(len(lem_dataset)):
    stm_dataset.iloc[index]['step'] = [' '.join([stemmer.stem(w) for w in word_tokenize(sent)]) for sent in lem_dataset.iloc[index]['step']]

Stampa dei risultati delle operazioni di stop word removal, stemming e lemming

In [ ]:
print("Frasi originali\n")
print(dataset.iloc[10]['step'])
print("\nStop word removal\n")
print(swr_dataset.iloc[10]['step'])
print("\nStop word e lemming\n")
print(lem_dataset.iloc[10]['step'])
print("\nStop word lemming e stemming\n")
print(stm_dataset.iloc[10]['step'])

### Entityt Extraction, Relation Extraction e POS tagging con SpaCy

In [ ]:
spacy_dataset = dataset.copy(deep=True)
       
for index in range(len(dataset)):
    spacy_dataset.iloc[index]['ingredients'] = [NLP(element) for element in dataset.iloc[index]['ingredients']]
    spacy_dataset.iloc[index]['step'] = [NLP(element) for element in dataset.iloc[index]['step']]

In [ ]:
def __pprintsp__(dataset, index, column, value, extend=False):
    displacy.render(dataset.iloc[index][column][value], style='dep')
    print(dataset.iloc[index][column][value])
    displacy.render(dataset.iloc[index][column][value], style='ent')
    if extend:
        for token in dataset.iloc[index][column][value]:
            print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)

def pprint_spacyd_all(dataset, index, column, extend=False):
    for v in range(len(dataset.iloc[index][column])):
        __pprintsp__(dataset, index, column, v, extend)
            
def pprint_spacyd(dataset, index, column, value, extend=False):
    __pprintsp__(dataset, index, column, value, extend)

### Analisi degli ingredienti

In [ ]:
class Recipe:
    def __init__(self, title, index, ingredients_str, steps_str):
        self.title = title
        self.ingredients = []
        self.ing_str = ingredients_str
        self.stp_str = steps_str
        self.steps = []
        self.idx = index
    
    def add_ing(self, ing):
        self.ingredients.append(ing)
        
    def __str__(self):
        _ing_s = '\n'.join([ing.short_srt() for ing in self.ingredients])
        _stp_s = '\n'.join(str(step) for step in self.steps)
        _header_ing = f"|{'NOME':<40s}|{'PROPRIETÀ':<20s}|{'QUANTITÀ':<11s}|{'TIPO':<21s}|"
        _header_ing_l = f"+{'-'*40}+{'-'*20}+{'-'*11}+{'-'*21}+"
        _header_step = f"|{'N°':<2s}|{'AZIONE':<15s}|{'STRUMENTI':<20s}|{'INGREDIENTI':<55s}|"
        _header_step_l = f"+{'-'*2}+{'-'*15}+{'-'*20}+{'-'*55}|"
        return f"TITOLO: {self.title}\n\n{_header_ing}\n{_header_ing_l}\n{_ing_s}\n{_header_ing_l}\n\n{_header_step}\n{_header_step_l}\n{_stp_s}\n{_header_step_l}"
    
    def print_unstructured(self):
        return f"{self.title}\n\n{self.ing_str}\n\n{self.stp_str}"
    
    
class Step:
    def __init__(self, step_str, step_no, action, ins, ing):
        self.action = action
        self.ins = ins
        self.ing = ing
        self.step_str = step_str
        self.step_no = step_no
        
    def __str__(self):
        return f"|{self.step_no:<2d}|{self.action:<15s}|{','.join(self.ins):20s}|{','.join(self.ing):55s}|"

    
    
class Ingredient:
    def __init__(self, full_text):
        self.name=""
        self.ing_cat_id = None
        self.size=""
        self.quantity= 0.0
        self.size=""
        self.adj=[]
        self.original=full_text
    def __str__(self):
        return f"Name: {self.name}\nAjdectives: {self.adj}\nQuantity: {self.quantity:4f} {self.size}\nOriginal: {self.original}"

    def short_srt(self):
        return f"|{self.name:<40s}|{','.join(self.adj):<20s}|{self.quantity:^11.2f}|{self.size:<21s}|"
    
    def set_id(self, identifier):
        self.ing_cat_id = identifier

Crezione elenco ingredienti e ricette

In [ ]:

def is_food(word):
    syns = wn.synsets(str(word), pos = wn.NOUN)
    for syn in syns:
        if 'food' in syn.lexname():
            return True
    return False

def cls_w(word):
    return word.strip().lower()

def parse_quantity(qt):
    if qt == "":
        return 1
    
    partial_sum = 0.0
    backtrack_multiplier = 1.0
    
    for elem in qt.split(" "):
        try:
        
            if ('/' in elem) and (partial_sum == 0.0):
                parsed = elem.split("/")
                backtrack_multiplier = float(parsed[0])/float(parsed[1])

            elif ('/' in elem) and (partial_sum != 0.0):
                parsed = elem.split("/")
                partial_sum += (float(parsed[0])/float(parsed[1]))

            else:
                partial_sum += float(elem)
        except ValueError:
            pass
            
    if(partial_sum != 0.0):
        partial_sum *= backtrack_multiplier
    else:
        partial_sum = backtrack_multiplier
        
    return partial_sum
            
        

#Root generation

__SIZES__ = [
       'package',
       'tablespoon',
       'number',
       'pint',
       'number',
       'gallon',
       'tablespoon',
       'square',
       'ounce',
       'pound',
       'quart',
       'cup',
       'teaspoon',
       'can'
]


all_recipes = []
    
for index in range(len(spacy_dataset)):
    
    recipe = Recipe(
        spacy_dataset.iloc[index]['title'], 
        index, 
        " ".join(dataset.iloc[index]['ingredients']),
        " ".join(dataset.iloc[index]['step'])       
    )
    
    for element in spacy_dataset.iloc[index]['ingredients']:

        ingredient = element

        IGN = Ingredient(ingredient.text)
        ROOT_NODE = [token for token in ingredient if token.dep_ == 'ROOT'][0]
        

        if (ROOT_NODE.pos == VERB):
                IGN.adj.append(cls_w(ROOT_NODE.text))
        else:
                IGN.name = cls_w(ROOT_NODE.text)
                
        stack = [element for element in ROOT_NODE.children]

        unparsed_quantity = ""
        
        while len(stack)!=0:
            CURRENT_NODE = stack.pop()

            if (CURRENT_NODE.pos == X) or (CURRENT_NODE.pos == NUM):
                    unparsed_quantity = CURRENT_NODE.text + " " + unparsed_quantity
            else:
                if CURRENT_NODE.text.lower() in  __SIZES__:
                    IGN.size += " " + cls_w(CURRENT_NODE.text)

                if ((CURRENT_NODE.dep_ == 'compound') or (CURRENT_NODE.dep_ == 'dobj') or (CURRENT_NODE.dep_ == 'pobj')) and is_food(CURRENT_NODE.text) and (CURRENT_NODE.text.lower() not in  __SIZES__):
                    IGN.name = CURRENT_NODE.text + " " + cls_w(IGN.name)

                if (CURRENT_NODE.dep_ == 'amod') or (CURRENT_NODE.dep_ == 'appos'):
                    if is_food(CURRENT_NODE.text):
                        IGN.name = CURRENT_NODE.text + " " + cls_w(IGN.name)
                    else:
                        IGN.adj.append(cls_w(CURRENT_NODE.text))         

            stack += [element for element in CURRENT_NODE.children]
        
        if IGN.name.strip() != "":
            IGN.quantity = parse_quantity(cls_w(unparsed_quantity))
            IGN.name = cls_w(IGN.name)
            recipe.add_ing(IGN)
    all_recipes.append(recipe)
        


### Analisi degli step

In [ ]:
ingredients_all_words = set()

for element in all_recipes:
    for ingredient in element.ingredients:
        for e in ingredient.name.split(" "):
            ingredients_all_words.add(cls_w(e))

ingredients_all_words.add('potato')

In [ ]:
def hidden_verb(word):
    ss = wn.synsets(word)
    for s in ss:
        if s.pos() == 'v':
            return True
    return False

def combine_action(action_token):
    action = cls_w(action_token.text)
    for child in action_token.children:
        if child.dep_ == 'prt' or child.dep_ == 'compound':
            action += " " + cls_w(child.text)
    return action




def find_instrument(token):
    stack = [element for element in token.children]
    instrument = ""
    found_main = False
    main_node = None

    while stack:
        s_element = stack.pop()
        if s_element.pos == NOUN and (cls_w(s_element.text) not in ingredients_all_words) and (not found_main) and (s_element.text not in forbidden_instruments):
            if "ed" != s_element.text[-2:] and "ing" != s_element.text[-3:]:
                instrument += cls_w(s_element.text)
                found_main = True
                main_node = s_element
        elif s_element.dep_ == 'compound' and main_node == s_element.head and (cls_w(s_element.text) not in ingredients_all_words):
            instrument = cls_w(s_element.text) + " " + instrument
            main_node = s_element
            
        stack += [element for element in s_element.children]
    return instrument


forbidden_actions = ['done', 'will', 'be']
forbidden_instruments = ['top']
PREPS = ['with', 'in', 'on', 'of', 'using', 'into', 'use']

def get_ingredients(recipe):
    return {ing.name for ing in recipe.ingredients}

def find_ingredients(ings, step):
     
    ing = set()
    for word in step.replace(',','').replace('.','').replace('(','').replace(')','').replace('*','').split(" "):
        word = cls_w(word)
        for el in ings:
            if re.findall(f"(\\s{word}\\s)|({word}\\s)|(\\s{word})", el) or el == word:
                ing.add(el)                
    return ing


for index in range(len(dataset)):
    step_counter = 1
    ing_s = get_ingredients(all_recipes[index])
    for value in range(len(dataset.iloc[index]['step'])):
        for element in dataset.iloc[index]['step'][value].split(";"):
               
            step = NLP(element.replace("°", '').strip())
            
            FINAL_ACTION = ""
            _hidden_verbs = []
            _found_action = False
            found_instruments = []
            found_ingredients = []
            
            for TOKEN in step:
                if ("ed" != TOKEN.text[-2:]) and TOKEN.text not in forbidden_actions and ("ing" != TOKEN.text[-3:]):
                    if not _found_action:
                        if (TOKEN.pos == VERB):
                            FINAL_ACTION = combine_action(TOKEN)
                            _found_action = True
                        elif hidden_verb(TOKEN.text):
                            _hidden_verbs.append(combine_action(TOKEN))
                            
                if TOKEN.text in PREPS:
                    instrument = find_instrument(TOKEN)
                    if instrument != "":
                        found_instruments.append(cls_w(instrument))
                        
                      
            if FINAL_ACTION == "" and _hidden_verbs:
                FINAL_ACTION = _hidden_verbs[0]
                
            if FINAL_ACTION != "":
                found_step = Step(element, step_counter, cls_w(FINAL_ACTION), found_instruments, find_ingredients(ing_s, step.text))
                all_recipes[index].steps.append(found_step)
                step_counter += 1

            

### Dump
Tutte le ricette vengono stampate in formato testuale secondo suddivisione in ingredienti, strumenti e step

In [ ]:
with open('../dump.txt', 'w') as f:
    for i, index in enumerate(all_recipes):
        f.write(index.print_unstructured() + '\n')
        f.write(f"[{i}]" + str(index) + '\n')

## GENERAZIONE DATABASE

Crezione indici per ricette e ingredienti

In [ ]:
macro_ingredients = {}
ingredients_idx = {}
macro_actions = {}
actions_idx = {}
macro_instruments = {}
instruments_idx = {}
##
actions_idx = {}

for element in all_recipes:
    for ingredient in element.ingredients:
        if ingredient.name in macro_ingredients:
            macro_ingredients[ingredient.name] += 1
        else:
            macro_ingredients[ingredient.name] = 1
            
for index, key in enumerate(macro_ingredients):
    ingredients_idx[key] = index
    #print(f"{key:>50s} {macro_ingredients[key]:<3d}")


for element in all_recipes:
    for step in element.steps:
        if step.action in macro_actions:
            macro_actions[step.action] += 1
        else:
            macro_actions[step.action] = 1
            
        for ins in step.ins:
            if ins in macro_instruments:
                macro_instruments[ins] += 1
            else:
                macro_instruments[ins] = 1
            
            
#print(100*'-')

for index, key in enumerate(macro_actions):
    actions_idx[key] = index
    #print(f"{key:>50s} {macro_steps[key]:<3d}")
    
#print(100*'-')

for index, key in enumerate(macro_instruments):
    instruments_idx[key] = index
    #print(f"{key:>50s} {macro_instruments[key]:<3d}")
    
#print(instruments_idx)
#print(steps_idx)
#print(ingredients_idx)
#print(actions_idx)
    
        
        


Creazione DDL ingredienti

In [ ]:
ING_DB_CREATE = """
USE recipe_analysis;
DROP TABLE IF EXISTS ingredients;
CREATE TABLE ingredients(
id INT PRIMARY KEY,
name VARCHAR(100) NOT NULL
);
"""

print(ING_DB_CREATE)


with open(Path("../database/create_ingredients.sql").resolve(), "w") as ddl_file:
    for ing in ingredients_idx:
        ING_DB_CREATE = ING_DB_CREATE + f"INSERT INTO ingredients VALUES ({ingredients_idx[ing]},\"{ing}\");\n"
    ddl_file.write(ING_DB_CREATE)

Creazione DDL strumenti

In [ ]:
ING_DB_CREATE = """
USE recipe_analysis;
DROP TABLE IF EXISTS instruments;
CREATE TABLE instruments(
id INT PRIMARY KEY,
name VARCHAR(100) NOT NULL
);
"""

print(ING_DB_CREATE)


with open(Path("../database/create_instruments.sql").resolve(), "w") as ddl_file:
    for ins in instruments_idx:
        ING_DB_CREATE = ING_DB_CREATE + f"INSERT INTO instruments VALUES ({instruments_idx[ins]},\"{ins}\");\n"
    ddl_file.write(ING_DB_CREATE)

Creazione DDL azioni

In [ ]:
ING_DB_CREATE = """
USE recipe_analysis;
DROP TABLE IF EXISTS actions;
CREATE TABLE actions(
id INT PRIMARY KEY,
name VARCHAR(100) NOT NULL
);
"""

print(ING_DB_CREATE)


with open(Path("../database/create_actions.sql").resolve(), "w") as ddl_file:
    for act in actions_idx:
        ING_DB_CREATE = ING_DB_CREATE + f"INSERT INTO actions VALUES ({actions_idx[act]},\"{act}\");\n"
    ddl_file.write(ING_DB_CREATE)

Creazione DDL ricette

In [ ]:
REC_DB_CREATE = """
USE recipe_analysis;
DROP TABLE IF EXISTS recipes;
CREATE TABLE recipes(
id INT PRIMARY KEY,
titolo VARCHAR(100) NOT NULL,
preparazione_nstr VARCHAR(500),
ingredienti_nstr VARCHAR(500)
);
"""

print(REC_DB_CREATE)



with open(Path("../database/create_recipes.sql").resolve(), "w") as ddl_file:
    for rec in all_recipes:
        
        corrected_ing = rec.ing_str.replace("\'","\'\'")
        corrected_stp = rec.stp_str.replace("\'","\'\'")
        corrected_tit = rec.title.replace("\'","\'\'")
        
        REC_DB_CREATE = REC_DB_CREATE + f"INSERT INTO recipes (id, titolo, ingredienti_nstr, preparazione_nstr) VALUES ({rec.idx},\'{corrected_tit}\',\'{corrected_ing}\',\'{corrected_stp}\');\n"
    ddl_file.write(REC_DB_CREATE)

Crezione DDL tabella relazione Ricette <--> Ingredienti

In [ ]:
CON_DB_CREATE = """
USE recipe_analysis;
DROP TABLE IF EXISTS contains;
CREATE TABLE contains(
id INT NOT NULL AUTO_INCREMENT,
misura VARCHAR(50),
quantita FLOAT,
proprieta VARCHAR(100),
recipeID int NOT NULL,
ingredientID int NOT NULL,
PRIMARY KEY (id),
FOREIGN KEY (recipeID) REFERENCES recipes(id),
FOREIGN KEY (ingredientID) REFERENCES ingredients(id)
);
"""

print(CON_DB_CREATE)


with open(Path("../database/create_contains.sql").resolve(), "w") as ddl_file:
    for r in all_recipes:
        for i in r.ingredients:
            if i.name.strip() != "":
                adj = ', '.join(i.adj)
                data = f"(\'{i.size}\',{i.quantity},\'{adj}\',{r.idx}, {ingredients_idx[i.name]})"
                CON_DB_CREATE = CON_DB_CREATE + f"INSERT INTO contains (misura, quantita, proprieta, recipeID, ingredientID) VALUES {data};\n"
    ddl_file.write(CON_DB_CREATE)      

Creazione DDL step

In [ ]:
CON_DB_CREATE = """
USE recipe_analysis;
DROP TABLE IF EXISTS steps;
CREATE TABLE contains(
id INT NOT NULL AUTO_INCREMENT,
descrizione VARCHAR(500),
numero_step INT,
PRIMARY KEY (id),
FOREIGN KEY (actionID) REFERENCES actions(id)
);
"""

print(CON_DB_CREATE)

with open(Path("../database/create_steps.sql").resolve(), "w") as ddl_file:
    for r in all_recipes:
        for s in r.steps:
            data = f"(\'{s.step_str}\',{s.step_no}, {actions_idx[s.action]})"
            CON_DB_CREATE = CON_DB_CREATE + f"INSERT INTO steps (descrizione, numero_step) VALUES {data};\n"
    ddl_file.write(CON_DB_CREATE)   

Crezione DDL tabella relazione Step <--> Strumenti

In [43]:
CON_DB_CREATE = """
USE recipe_analysis;
DROP TABLE IF EXISTS utilize;
CREATE TABLE utilize(
id INT NOT NULL AUTO_INCREMENT,
PRIMARY KEY (id),
FOREIGN KEY (instrumentID) REFERENCES instruments(id),
FOREIGN KEY (stepID) REFERENCES steps(id)
);
"""

print(CON_DB_CREATE)


with open(Path("../database/create_utilize.sql").resolve(), "w") as ddl_file:
    for r in all_recipes:
        for s in r.steps:
            for i in s.ins:
                data = f"({instruments_idx[i]}, {s.step_no})"
                CON_DB_CREATE = CON_DB_CREATE + f"INSERT INTO utilize (instrumentID, stepID) VALUES {data};\n"
    ddl_file.write(CON_DB_CREATE)  


USE recipe_analysis;
DROP TABLE IF EXISTS utilize;
CREATE TABLE utilize(
id INT NOT NULL AUTO_INCREMENT,
PRIMARY KEY (id),
FOREIGN KEY (instrumentID) REFERENCES instruments(id),
FOREIGN KEY (stepID) REFERENCES steps(id)
);



Crezione DDL tabella relazione Step <--> Ricette

In [44]:
CON_DB_CREATE = """
USE recipe_analysis;
DROP TABLE IF EXISTS composed;
CREATE TABLE composed(
id INT NOT NULL AUTO_INCREMENT,
PRIMARY KEY (id),
FOREIGN KEY (recipeID) REFERENCES recipes(id),
FOREIGN KEY (stepID) REFERENCES steps(id)
);
"""

print(CON_DB_CREATE)


with open(Path("../database/create_composed.sql").resolve(), "w") as ddl_file:
    for r in all_recipes:
        for s in r.steps:
            data = f"({r.idx}, {s.step_no})"
            CON_DB_CREATE = CON_DB_CREATE + f"INSERT INTO composed (recipeID, stepID) VALUES {data};\n"
    ddl_file.write(CON_DB_CREATE)  


USE recipe_analysis;
DROP TABLE IF EXISTS composed;
CREATE TABLE composed(
id INT NOT NULL AUTO_INCREMENT,
PRIMARY KEY (id),
FOREIGN KEY (recipeID) REFERENCES recipes(id),
FOREIGN KEY (stepID) REFERENCES steps(id)
);



Crezione DDL tabella relazione Step <--> Ingredienti

In [45]:
CON_DB_CREATE = """
USE recipe_analysis;
DROP TABLE IF EXISTS involves;
CREATE TABLE involves(
id INT NOT NULL AUTO_INCREMENT,
PRIMARY KEY (id),
FOREIGN KEY (ingredientID) REFERENCES ingredients(id),
FOREIGN KEY (stepID) REFERENCES steps(id)
);
"""

print(CON_DB_CREATE)


with open(Path("../database/create_involves.sql").resolve(), "w") as ddl_file:
    for r in all_recipes:
        for s in r.steps:
            for i in s.ing:
                data = f"({ingredients_idx[i]}, {s.step_no})"
                CON_DB_CREATE = CON_DB_CREATE + f"INSERT INTO involves (ingredientID, stepID) VALUES {data};\n"
    ddl_file.write(CON_DB_CREATE)  


USE recipe_analysis;
DROP TABLE IF EXISTS involves;
CREATE TABLE involves(
id INT NOT NULL AUTO_INCREMENT,
PRIMARY KEY (id),
FOREIGN KEY (ingredientID) REFERENCES ingredients(id),
FOREIGN KEY (stepID) REFERENCES steps(id)
);

